# Example: Multiple Tables
In this example, I show how doctable can be used with multiple inter-related tables to perform queries which automatically merge different aspects of your dataset when you use `.select()`. By integrating these relations into the schema, your database can automatically maintain consistency between tables by deleting irrelevant elements when their relations disappear.

For this example, we create a doctable for books called `Books` a authors called `AuthorTable` containing _elements_ that can each have a number of _sub-elements_ in `SubTable`. In this design, each sub-element in `SubTable` should maintain what is called a _foreign key_ reference to the primary key column in `MainTable`.

In [1]:
import sys
sys.path.append('..')
import doctable
class Authors(doctable.DocTable):
    tabname = 'authors'
    schema = (
        ('idcol', 'id'),
        ('string', 'name'),
        ('date', 'birthday'),
        ('date_updated', 'updated'),
    )
db = Authors()
db.schematable

class Books(doctable.DocTable):
    tabname = 'books'
    schema = (
        ('idcol', 'id'), # each book has its own id
        ('string', 'title'),
        ('integer', 'authid'), # reference to author table entry
        ('foreignkey', 'authid', 'authors.id'),
    )
bt = Books(engine=db._engine)
bt


,name,type,nullable,default,autoincrement,primary_key
0,id,INTEGER,False,None,auto,1
1,name,VARCHAR,True,None,auto,0
2,birthday,DATE,True,None,auto,0
3,updated,DATETIME,True,None,auto,0


First, we create a doctable called `AuthorTable` to store information about authors.

NoReferencedTableError: Foreign key associated with column 'books.authid' could not find table 'authors' with which to generate a foreign key to target column 'id'

Next, we create a class MainTable with a foreign key constraint. The coreign key points from it's own column subkey to the SubTab `id` column. Additionally, we overload the MainTable insert method to store the provided `n` value into the other table and instead insert a reference to the SubTable that stores the actual integer.

Now we test our code by creating a new MainTable object (which also instantiates a SubTable as a member variable), and then inserts two rows with different names and the same number. The result is that two rows have been inserted in MainTable but they contain references to the saim subtable rows.

In [5]:
db = MainTable('exdb/multitable0.db')
print(db)
for name in ('devin cornell', 'pierre bourdieu'):
    db.insert({'name':name, 'n':10})
print(db)
print(db.select_df())
print(db.subtab.select_df())

<DocTable::maintable ct: 0>
<DocTable::maintable ct: 2>
   id  subkey             name                    updated
0   1       1    devin cornell 2020-01-31 19:13:15.624948
1   2       1  pierre bourdieu 2020-01-31 19:13:15.628480
   id   n                    updated
0   1  10 2020-01-31 19:13:15.552712


Note how the output shows two rows created in the MainTable but only one in the SubTable. According to computational constraints one might choose not to perform the checks for table existance in the insert method, but in this case it is useful.

## Automatically Join Using Select
The great part about using sqlalchemy core is that you can manipulate multiple tables by accessing their doctable objects. In this case, we perform a join of name to their associated n (stored in the subtable) using the column subscript for the db.subtab member (of type SubTable). By default this does an outer join! Not sure what happens in a many-to-many table.

In [9]:
db.select(['name', db.subtab['n']])

[('devin cornell', 10), ('pierre bourdieu', 10)]